In [135]:
from llama_index.core import SimpleDirectoryReader
import os
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core import Settings
import torch
#from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

#from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.storage.docstore import SimpleDocumentStore
#from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.extractors import TitleExtractor



In [111]:
print(f"Chunk Size: {Settings.chunk_size}")

Chunk Size: 512


In [149]:
# Load documents from the directory
reader = SimpleDirectoryReader(input_dir="C:\\Users\\natuk\\Documents\\Masters\\LLama",recursive = True)

#C:\\Users\\natuk\\Documents\\Masters\\LLama
#C:\\Users\\natuk\\Documents\\GitHub\\Smart-Tutor-AI-AI-Driven-Personalized-Teaching-Support_origin\\data\\module_5
documents = reader.load_data()


model.safetensors:   0%|          | 0.00/982M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.49.0"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id"

In [154]:
import os
import re


# Define the preprocessing function
def preprocess_text(file_path, text):
    """
    Preprocesses the text depending on the file type:
    - Removes punctuation for non-code files
    - Leaves code files untouched
    """
    # Define file extensions for code and text
    code_extensions = {".py", ".ipynb"}
    text_extensions = {".pdf", ".docx", ".pptx", ".txt"}

    # Check the file extension
    ext = os.path.splitext(file_path)[-1].lower()

    # If it's a text file, remove punctuation
    if ext in text_extensions:
       # Preserve email addresses by replacing them with a placeholder
        text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', 'EMAIL_PLACEHOLDER', text)
        
        # Remove punctuation and unnecessary whitespace
        text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Keep words and spaces only
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
        text = re.sub(r"\b\d{1,2}/\d{1,2}/\d{4}\b", "", text)  # Remove dates like 3/3/2025
        text = re.sub(r"\b\d{5,}\s\d+\b", "", text)  # Remove page numbers like 332025 54
        text = re.sub(r'\b[A-Za-z0-9+/]{20,}={0,2}\b', '', text)  # Remove base64 (image data)
        
        # Restore email addresses
        text = text.replace('EMAIL_PLACEHOLDER', 'example@example.com')  # You can adjust this to keep actual email addresses if needed
    # If it's a code file (e.g., Python), leave the text as is
    elif ext in code_extensions:
        text = re.sub(r'\b[A-Za-z0-9+/]{20,}={0,2}\b', '', text)  # Remove base64 (image data)  # Do not modify code files

    return text

'''
# Apply metadata to each document and preprocess text
for doc in documents:
    doc.metadata = {
        "file_name": doc.metadata.get("file_name", ""),
        "file_path": doc.metadata.get("file_path", ""),
        "num_tokens": len(doc.get_content().split()),  # Estimate token count
        "num_chars": len(doc.get_content()),  # Count characters
    }
'''
for doc in documents:
    file_path = doc.metadata.get("file_path", "")
    
    # Extract folder name from file_path
    folder_name = os.path.basename(os.path.dirname(file_path)) if file_path else ""

    doc.metadata = {
        "file_name": doc.metadata.get("file_name", ""),
        "file_path": file_path,
        "folder_name": folder_name,  # Include folder name in metadata
        "num_tokens": len(doc.get_content().split()),  # Estimate token count
        "num_chars": len(doc.get_content()),  # Count characters
    }
    
    # Preprocess the document's text and update content using set_content()
    doc_text = doc.get_content()  # Use get_content() to retrieve text
    processed_text = preprocess_text(doc.metadata["file_path"], doc_text)
    doc.set_content(processed_text)  # Use set_content() to update text








In [152]:
distinct_files = set()

for doc in documents:
    file_path = doc.metadata.get("file_path", "Unknown file")
    distinct_files.add(file_path)

# Print distinct file paths
for file in distinct_files:
    print(file)


C:\Users\natuk\Documents\Masters\LLama\Module1\Lesson two-Python Basic-Fall2024.pptx
C:\Users\natuk\Documents\Masters\LLama\Module1\Week_3_Demo.ipynb
C:\Users\natuk\Documents\Masters\LLama\Module2\INFO5731_Syllabus_Sping2025_updated_01142025.pdf
C:\Users\natuk\Documents\Masters\LLama\Module2\Lesson_three_Python_Basic.pptx


In [155]:
for i, doc in enumerate(documents):
    print(f"\nDocument {i + 1}:")
    print(f"Document ID: {doc.doc_id}")  # Unique identifier for the document
    print(doc.metadata)
    print(f"Text: {doc.text[:500]}...")


Document 1:
Document ID: 1114490c-ef9c-4b40-a808-e691c7699a4f
{'file_name': 'Lesson two-Python Basic-Fall2024.pptx', 'file_path': 'C:\\Users\\natuk\\Documents\\Masters\\LLama\\Module1\\Lesson two-Python Basic-Fall2024.pptx', 'folder_name': 'Module1', 'num_tokens': 2168, 'num_chars': 11772}
Text: Slide 0 Image a large group of birds sitting on top of a building Lesson 2 Python Basic 1 Haihua Chen PhD Slide 1 1 2 3 4 Image a city with tall buildings and a sky background Basic Syntax Python Variables Python Operators Python Data Types 5 Python Files 6 Loop 7 Conditionals 8 Inclass Quiz 1 Slide 2 Image a large group of birds sitting on top of a building 1 Basic Syntax Slide 3 a highlevel programming language like Java C C et al 1 Execute Python Syntax Interpreter immediate mode and script m...

Document 2:
Document ID: 13e77c76-7208-4a03-a17b-2016aae2229f
{'file_name': 'Week_3_Demo.ipynb', 'file_path': 'C:\\Users\\natuk\\Documents\\Masters\\LLama\\Module1\\Week_3_Demo.ipynb', 'folder_name

In [156]:

'''splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95
)

# also baseline splitter
base_splitter = SentenceSplitter(chunk_size=512)


nodes = splitter.get_nodes_from_documents(documents)'''
# Define chunk size (set a fixed value, e.g., 300 characters)
CHUNK_SIZE = 512
CHUNK_OVERLAP = 5  # Optional: overlap between chunks for context

# Function to split text into fixed-size chunks
def chunk_text(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i : i + chunk_size])
    return chunks

# Process each document
processed_docs = []
for doc in documents:
    text_chunks = chunk_text(doc.get_content())  # Split document text into chunks
    for chunk in text_chunks:
        file_path = doc.metadata.get("file_path", "")
    
    # Extract folder name from file_path
        folder_name = os.path.basename(os.path.dirname(file_path)) if file_path else ""

        metadata = {
            "file_name": doc.metadata.get("file_name", ""),
            "file_path": file_path,
            "folder_name": folder_name,  # Include folder name in metadata
            "num_tokens": len(chunk.split()),  # Estimate tokens in each chunk
            "num_chars": len(chunk),  # Ensure each chunk has a fixed number of characters
        }
        processed_docs.append({"doc_id": doc.doc_id, "text": chunk, "metadata": metadata})

# Print the processed documents with fixed-size chunks
for i, doc in enumerate(processed_docs):
    print(f"\nChunk {i + 1}:")
    print(f"Document ID: {doc['doc_id']}")
    print(f"Metadata: {doc['metadata']}")
    print(f"Text: {doc['text']}\n")


Chunk 1:
Document ID: 1114490c-ef9c-4b40-a808-e691c7699a4f
Metadata: {'file_name': 'Lesson two-Python Basic-Fall2024.pptx', 'file_path': 'C:\\Users\\natuk\\Documents\\Masters\\LLama\\Module1\\Lesson two-Python Basic-Fall2024.pptx', 'folder_name': 'Module1', 'num_tokens': 99, 'num_chars': 512}
Text: Slide 0 Image a large group of birds sitting on top of a building Lesson 2 Python Basic 1 Haihua Chen PhD Slide 1 1 2 3 4 Image a city with tall buildings and a sky background Basic Syntax Python Variables Python Operators Python Data Types 5 Python Files 6 Loop 7 Conditionals 8 Inclass Quiz 1 Slide 2 Image a large group of birds sitting on top of a building 1 Basic Syntax Slide 3 a highlevel programming language like Java C C et al 1 Execute Python Syntax Interpreter immediate mode and script mode Image a 


Chunk 2:
Document ID: 1114490c-ef9c-4b40-a808-e691c7699a4f
Metadata: {'file_name': 'Lesson two-Python Basic-Fall2024.pptx', 'file_path': 'C:\\Users\\natuk\\Documents\\Masters\\LLama\\M

In [161]:
with open("parsedData.txt", "w", encoding="utf-8") as f:
    for i, doc in enumerate(processed_docs):
        # Writing the chunk number
        f.write(f"Chunk {i + 1}:\n")
        
        # Writing the Document ID
        f.write(f"Document ID: {processed_docs.doc_id}\n")
        
        # Writing the Metadata
        metadata = processed_docs.metadata if processed_docs.metadata else {}
        f.write(f"Metadata: {metadata}\n")
        
        # Writing the Text
        f.write(f"Text: {processed_docs.text}\n\n")


AttributeError: 'list' object has no attribute 'doc_id'

In [65]:
distinct_files = set()

for doc in documents:
    file_path = doc.metadata.get("file_path", "Unknown file")
    distinct_files.add(file_path)

# Print distinct file paths
for file in distinct_files:
    print(file)


C:\Users\natuk\Documents\Masters\LLama\Demo01_Evidently_Metrics.ipynb
